In [1]:
import duckdb
con = duckdb.connect()

In [2]:
def cria_tab(con):
    # Função para criação de tabelas no duckdb
    con.sql("DROP TABLE IF EXISTS Usina; DROP SEQUENCE IF EXISTS id_usina; DROP TABLE IF EXISTS Precipitacao;")
    sql_create_usina = """
        CREATE SEQUENCE id_usina;
        CREATE TABLE IF NOT EXISTS Usina (
            id INTEGER PRIMARY KEY,
            nome VARCHAR NOT NULL,
            proprietario VARCHAR,
            munic1 VARCHAR NOT NULL,
            uf1 VARCHAR NOT NULL,
            munic2 VARCHAR,
            uf2 VARCHAR,
            rio VARCHAR NOT NULL,
            potkw FLOAT,
            atualizacao DATE,
            iniciop DATE,
            potfisc FLOAT,
            lat DOUBLE PRECISION,
            long DOUBLE PRECISION,
            estagio VARCHAR
        );
    """
    con.sql(sql_create_usina)
    con.commit()

def carrega_csv(con):
    # Função para carregar tabelas com os dados puxados dos arquivos .csv na pasta Datasets
    sql_carrega_tabela_usi = """
        INSERT INTO Usina (id, nome, proprietario, munic1, uf1, munic2, uf2, rio, potkw, atualizacao, iniciop, potfisc, lat, long, estagio)
        SELECT nextval('id_usina') as id, nome, prop as proprietario, munic1, uf1, munic2, uf2, rio,
        CAST(REPLACE(CAST(potkw AS VARCHAR), ',', '.') AS DOUBLE),
        TRY_CAST(atualiz AS DATE) as atualizacao,
        CASE WHEN iniciop = '0' THEN NULL ELSE STRPTIME(iniciop, '%d/%m/%Y') END as iniciop,
        CAST(REPLACE(CAST(potfisc AS VARCHAR), ',', '.') AS DOUBLE),
        CAST(REPLACE(lat, ',', '.') AS DOUBLE),
        CAST(REPLACE(long, ',', '.') AS DOUBLE),
        estag as estagio
        FROM read_csv(['Datasets/Centrais_Geradoras_Hidrelétricas_CGH.csv', 'Datasets/Usinas_Hidrelétricas_UHE.csv'], union_by_name=true,sep=';', dateformat = '%d/%m/%Y')
    """
    sql_clean_usiprec = """DROP TABLE IF EXISTS Usi_Prec_Desnorm; DROP SEQUENCE IF EXISTS id_usi_prec;"""
    sql_create_desnorm_usiprec = """
        CREATE SEQUENCE id_usi_prec;
        CREATE TABLE IF NOT EXISTS Usi_Prec_Desnorm AS
        SELECT u.*, p.medanual, p.dpanual, p.cvanual, p.minanual, p.maxanual, p.ndiasecoscons, p.mxdiasecos, p.mndiasecos, p.nr20, p.mdr20, p.mxr20, p.prec_max_gev
        FROM Usina u
        JOIN Precipitacao p
        ON ABS(u.lat - p.lat) <= 0.08 AND ABS(u.long - p.long) <= 0.08
    """
    con.sql(sql_carrega_tabela_usi)
    con.commit()

def executa_query(con, query):
    print(con.sql(query).show())

In [3]:
cria_tab(con)
carrega_csv(con)

In [4]:
# Talvez aqui seja uma bom lugar para explicar o significa cada campo na saída das pesquisas
# id                - número inteiro único que identifica cada usina
# nome              - nome da usina
# proprietario      - empresa proprietária da usina
# munic1            - município principal no qual a usina está localizada
# uf1               - código da unidade federativa na qual a usina está localizada
# munic2            - segundo município afetado pela usina fora o município principal
# uf2               - unidade federativa em que está o segundo município
# rio               - rio no qual a usina foi instalada
# potkw             - potência em kilowatts obtida
# atualizacao       - última data de atualização de dados
# iniciop           - data de início das operações
# potfisc           - potência máxima fiscalizada
# lat               - latitude
# long              - longitude
# estagio           - classificação que abrange sua operação, desativação, eixo instalado mas inoperante, etc. 
con.sql("SELECT * FROM Usina").show()

┌───────┬───────────────────────────────────────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬───────────────────────┬─────────┬────────────────┬─────────┬───────────────────┬───────────┬─────────────┬────────────┬───────────┬──────────────┬──────────────┬─────────────────┐
│  id   │                     nome                      │                                                                          proprietario                                                                           │        munic1         │   uf1   │     munic2     │   uf2   │        rio        │   potkw   │ atualizacao │  iniciop   │  potfisc  │     lat      │     long     │     estagio     │
│ int32 │                    varchar                    │                                                                             varchar                                                           

In [5]:
# Porcentagem de usinas que abastecem dois estados e dois municípios e as maiores potências (no papel) dessas usinas
query_1A = """  WITH porcentagens AS ( 
                    SELECT (COUNT(CASE WHEN uf1 <> uf2 THEN 1 END) * 100.0 / COUNT(*)) AS porcentagem_multiestado, (COUNT(CASE WHEN munic1 <> munic2 THEN 1 END) * 100.0 / COUNT(*)) AS porcentagem_multimunicipio FROM Usina),
                maior_pot_multiestado AS ( SELECT MAX(potkw) AS maior_pot_teorica_multiestado
                    FROM Usina
                    WHERE uf1 <> uf2
                ),
                maior_pot_multimunicipio AS ( SELECT MAX(potkw) AS maior_pot_teorica_multimunicipio
                    FROM Usina
                    WHERE munic1 <> munic2
                )
                SELECT * FROM porcentagens
                CROSS JOIN maior_pot_multiestado
                CROSS JOIN maior_pot_multimunicipio;"""

# Porcentagem de usinas que abastecem dois municípios
# E dentre elas, quanto se resume a dois municípios no mesmo estado e quanto dois municípios em dois estados distintos



executa_query(con, query_1A)

┌─────────────────────────┬────────────────────────────┬───────────────────────────────┬──────────────────────────────────┐
│ porcentagem_multiestado │ porcentagem_multimunicipio │ maior_pot_teorica_multiestado │ maior_pot_teorica_multimunicipio │
│         double          │           double           │             float             │              float               │
├─────────────────────────┼────────────────────────────┼───────────────────────────────┼──────────────────────────────────┤
│                 20.3125 │                   53.90625 │                     3444000.0 │                        3444000.0 │
└─────────────────────────┴────────────────────────────┴───────────────────────────────┴──────────────────────────────────┘

None


In [6]:
# Gera um ranking dos estados, baseando-se em uma ESTIMATIVA de quanto cada uma de suas usinas ativas melhor
# aproveita seus potenciais em relação ao máximo fiscalizado.
# Como a proporção de divisão de energia para cada usina de dois estados é desconhecida, foi tomada a proporção 50/50
query_2A = """
    WITH temp AS (
      SELECT uf1 AS estado, potkw / 2 AS estado_potkw, potfisc / 2 AS estado_potfisc
      FROM Usina
      WHERE uf1 IS NOT NULL AND uf2 IS NOT NULL
        AND potfisc IS NOT NULL AND potfisc != 0 AND potkw IS NOT NULL AND potkw != 0
    
      UNION ALL
    
      SELECT uf2 AS estado, potkw / 2 AS estado_potkw, potfisc / 2 AS estado_potfisc
      FROM Usina
      WHERE uf1 IS NOT NULL AND uf2 IS NOT NULL
        AND potfisc IS NOT NULL AND potfisc != 0 AND potkw IS NOT NULL AND potkw != 0
    
      UNION ALL
    
      SELECT uf1 AS estado, potkw AS estado_potkw, potfisc AS estado_potfisc
      FROM Usina
      WHERE uf2 IS NULL
        AND potfisc IS NOT NULL AND potfisc != 0 AND potkw IS NOT NULL AND potkw != 0
    )

    SELECT
      estado,
      SUM(estado_potkw) AS potencial_usado,
      SUM(estado_potfisc) AS potencial_total,
      SUM(estado_potkw) / NULLIF(SUM(estado_potfisc), 0) AS aproveitamento
    FROM temp
    GROUP BY estado
    HAVING aproveitamento IS NOT NULL
    ORDER BY aproveitamento DESC;
"""

executa_query(con, query_2A)

┌─────────┬────────────────────┬────────────────────┬────────────────────┐
│ estado  │  potencial_usado   │  potencial_total   │   aproveitamento   │
│ varchar │       double       │       double       │       double       │
├─────────┼────────────────────┼────────────────────┼────────────────────┤
│ MG      │          2610440.0 │          2610440.0 │                1.0 │
│ MS      │          3267600.0 │          3267600.0 │                1.0 │
│ PR      │          1106246.5 │          1106296.5 │ 0.9999548041596443 │
│ SP      │ 10774918.419950008 │ 10786138.419950008 │ 0.9989597760047982 │
└─────────┴────────────────────┴────────────────────┴────────────────────┘

None


In [7]:
# Média de potência por década
query_3A = """ SELECT CAST(FLOOR(extract('year' FROM iniciop) / 10) * 10 AS INTEGER) || 's' AS decada, ROUND(AVG(potkw), 2) AS pot_med
            FROM Usina
            WHERE estagio = 'Operação'
            GROUP BY decada
            ORDER BY decada;"""

executa_query(con, query_3A)

┌─────────┬───────────┐
│ decada  │  pot_med  │
│ varchar │  double   │
├─────────┼───────────┤
│ 1900s   │    857.81 │
│ 1910s   │   7865.34 │
│ 1920s   │  87125.09 │
│ 1930s   │   1219.57 │
│ 1940s   │     332.5 │
│ 1950s   │  14908.52 │
│ 1960s   │ 239522.14 │
│ 1970s   │  638305.0 │
│ 1980s   │ 134543.67 │
│ 1990s   │ 411676.25 │
│ 2000s   │   11558.0 │
│ 2010s   │    1207.4 │
│ NULL    │       0.0 │
├─────────┴───────────┤
│ 13 rows   2 columns │
└─────────────────────┘

None


In [14]:
# Descobre os rios que geram mais potencial, suas quantidades respectivas de usinas e o estado principal que abastece
query_4B = """ SELECT rio, ROUND(SUM(potkw), 2) AS pot_total, COUNT(*) AS quantidade_usinas
        FROM Usina
        WHERE estagio = 'Operação'
        GROUP BY rio
        ORDER BY pot_total DESC;
        """
executa_query(con, query_4B)


┌────────────────────────┬───────────┬───────────────────┐
│          rio           │ pot_total │ quantidade_usinas │
│        varchar         │  double   │       int64       │
├────────────────────────┼───────────┼───────────────────┤
│ Rio Paraná             │ 6535200.0 │                 3 │
│ Rio Grande             │ 5220200.0 │                 7 │
│ Rio Paranapanema       │ 2410756.0 │                11 │
│ Rio Tietê              │ 1882050.0 │                 9 │
│ Rio Pedras             │  889000.0 │                 1 │
│ Rio Juquiá-Guaçu       │  230700.0 │                 6 │
│ Rio Pardo              │  222816.0 │                 4 │
│ Rio Paraibuna          │   87020.0 │                 1 │
│ Rio Sorocaba           │   61240.0 │                 4 │
│ Rio Paraíba do Sul     │   56050.0 │                 1 │
│         ·              │       ·   │                 · │
│         ·              │       ·   │                 · │
│         ·              │       ·   │                 ·

In [9]:
# Quantidade de usinas que não produzem por estado, e quanto de energia potencial é deixada de ser explorada por consequência
query_5A = """ SELECT uf1, COUNT(*) AS quantidade_inoperante
            FROM Usina
            WHERE estagio != 'Operação' OR potkw = 0 or potfisc = 0
            GROUP BY uf1;"""

executa_query(con, query_5A)

┌─────────┬───────────────────────┐
│   uf1   │ quantidade_inoperante │
│ varchar │         int64         │
├─────────┼───────────────────────┤
│ SP      │                    27 │
└─────────┴───────────────────────┘

None
